In [1]:
import polars as pl

In [2]:
# Load bike trips and weather
trips = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv", null_values='NA')
weather = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv", null_values=['NA',''])

trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d"))
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d"))

print(trips.height)
print(weather.height)

78704
733


In [3]:
# Question 1A: Join trips and weather
trips_weather = trips.join(weather, on="date", how="inner")

In [4]:
# Question 1B: Average duration of trips in bad weather
avg_bad_weather = trips_weather.filter(pl.col("events").is_not_null()).select(pl.col("duration").mean().alias("avg_duration"))
print(avg_bad_weather)

shape: (1, 1)
┌──────────────┐
│ avg_duration │
│ ---          │
│ f64          │
╞══════════════╡
│ 10.34899     │
└──────────────┘


In [5]:
# Load baseball datasets
Batting = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv", null_values='NA')
People = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv", null_values='NA')
HallOfFame = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv", null_values='NA')


In [6]:
# Question 2A: Join Batting and People
stats_w_bio = Batting.join(People, on="playerID", how="inner")

In [7]:
# Question 2B: Total home runs by Florida-born players
florida_hr = stats_w_bio.filter(pl.col("birthState")=="FL").select(pl.col("HR").sum().alias("total_hr"))
print(florida_hr)

shape: (1, 1)
┌──────────┐
│ total_hr │
│ ---      │
│ i64      │
╞══════════╡
│ 16225    │
└──────────┘


In [8]:
# Question 3A: Clean HallOfFame data
hall_inducted = HallOfFame.filter((pl.col("inducted")=="Y") & (pl.col("category")=="Player")).select("playerID").unique()

In [9]:
# Question 3B: Average career hits for Hall of Fame players
hof_hits = Batting.join(hall_inducted, on="playerID", how="inner") \
    .group_by("playerID").agg(pl.col("H").sum().alias("career_hits")) \
    .select(pl.col("career_hits").mean().alias("average_hits"))
print(hof_hits)

shape: (1, 1)
┌──────────────┐
│ average_hits │
│ ---          │
│ f64          │
╞══════════════╡
│ 1717.29918   │
└──────────────┘
